In [39]:
'''
# Input


# Input

1. wide data(date index 포함해야 함)
2. map data involves columns ticker, index

---
# Feature Extraction

map index에 따라
minus -> differ function
change -> returner function
else -> 그대로 

---
# ACF
acf_fal function
(cubic spline -> ffill -> bfill 고정 적용)

---
'''  

'\n# Input\n\n1. wide data involves date index\n2. map data involves columns ticker, index\n\n---\n# Feature Extraction\n\nmap index에 따라\nminus -> differ function\nchange -> returner function\nelse -> 그대로 \n\n---\n# ACF\nacf_fal function\n(cubic spline -> ffill -> bfill 고정 적용)\n\n\n---\n# Interpolation\n\n유형은 3가지로 \nstep, linear, spline\n\n---\n'

In [40]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
newdf = pd.read_csv('semiconductor_bloomberg_raw_20200109.csv', index_col = 0)

In [42]:
newdf.head()

,date,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,...,KOMSLYIndex,KOMSM1FYIndex,KOFETOTIndex,KOFETOT%Index,KORP7DIndex,KOCICOBAIndex,KOCICONFIndex,KRBO3MIndex,KWCDCCurncy,GVSK10YRIndex
0,1981-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1981-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1981-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1981-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1981-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
len(newdf.columns)

291

In [44]:
mmdf = pd.read_csv('bloomberg_map_index.csv')

In [45]:
mmdf.head()

,ticker,index
0,SPXIndex,change
1,CCMPIndex,change
2,NKYIndex,change
3,CINDIndex,change
4,IXKIndex,change


In [46]:
# date time으로 변경이 안되어 있었다 .
newdf['date'] = pd.to_datetime(newdf['date'])

In [47]:
newdf.set_index('date', inplace = True)

In [48]:
# 제대로 Column 들이 박혀있는 Dataframe이 들어오는 클래스
class Builder:

    def __init__(self, input_df, map_df):
        self.df = input_df
        self.mdf = map_df
        self.lag_days = [1, 7, 30, 90, 180, 365] # lagging day variation


    # df should involve datetime index
    # target column example = 'SKhynix_close'
    def returner(self, target_column):      

        col_name =  []
        # creating new column name
        for i in self.lag_days:
            col_name.append(target_column + str(i))
        # appending lagged columns
        for i, j in enumerate(self.lag_days):
            self.df[locals()['col_name'][i]] = self.df[target_column] / self.df[target_column].shift(periods = j, axis = 0)

        return self.df # Class 내에 남아있는 구조 여야 하는데 ..

    # df should involve datetime index
    # target column example = 'SKhynix_close'
    
    def differ(self, target_column):
        col_name = []
        for i in self.lag_days:
            col_name.append(target_column + str(i))
        for i, j in enumerate(self.lag_days):
            self.df[locals()['col_name'][i]] = self.df[target_column] - self.df[target_column].shift(periods = j, axis = 0)
        return self.df
    
    
    def execution(self):
        for i, j in enumerate(self.mdf['ticker']): # 키 값들을 순차적으로 돌리면서 순회, 예외사항이 있는지? 아 else를 걸어두면 상관없겠구나. 딱 mdf에 있는 만큼만 반영될듯
            #if self.mdf.loc[i, :][0] in self.df.columns:
            if j in self.df.columns:

                if self.mdf.loc[i, :][1] == 'minus':
                    self.differ(j)

                elif self.mdf.loc[i, :][1] == 'change':
                    self.returner(j)

                else:
                    pass
            else:
                pass
            
        return self.df
            
    # 차분에 ACF 그래프도 그리면 좋을듯. 정상성을 나타내지 않는 시계열에서는 ACF가 느리게 감소하고
    # 정상성을 나타내는 시계열에서는 ACF가 비교적 빠르게 0으로 떨어질 것
    # 구글 주식 가격 데이터의 경우 차분을 하면 ACF는 단순히 white noise time series 처럼 생겼음


    # spline이 된 상태에서는 acf를 적용할 수 있다. 
    # cubic spline만 적용한다고 가정하고
    # raw에서 채우지는 않고 acf만 보는 function을 짜자
    def acf_cal(self, target):
        data = pd.DataFrame(self.df[target])
        data = data.interpolate(method = 'cubic', limit_area = 'inside').fillna(method = 'ffill').fillna(method = 'bfill')
        acf = sm.tsa.acf(data)      
        plt.stem(acf)
        plt.show()



In [49]:
cla = Builder(newdf, mmdf)

In [50]:
cla.df.columns

Index(['GDPCQOQIndex', 'GDPCYOYIndex', 'GDPCTOT%Index', 'GPDITOC%Index',
       'GPGSTOC%Index', 'RGCDCIPIIndex', 'GDPCUR$Index', 'GDPCURYIndex',
       'GDPPIQQIndex', 'GDPCPCECIndex',
       ...
       'KOMSLYIndex', 'KOMSM1FYIndex', 'KOFETOTIndex', 'KOFETOT%Index',
       'KORP7DIndex', 'KOCICOBAIndex', 'KOCICONFIndex', 'KRBO3MIndex',
       'KWCDCCurncy', 'GVSK10YRIndex'],
      dtype='object', length=290)

In [51]:
cla.execution()

,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,...,KOFETOTIndex30,KOFETOTIndex90,KOFETOTIndex180,KOFETOTIndex365,KWCDCCurncy1,KWCDCCurncy7,KWCDCCurncy30,KWCDCCurncy90,KWCDCCurncy180,KWCDCCurncy365
date,,,,,,,,,,,,,,,,,,,,,
1981-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.11,-0.34,-0.45,-0.21
2019-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.08,-0.11,-0.29,-0.43,-0.21


In [52]:
len(cla.df.columns)

986

In [53]:
len(newdf.columns)

986

In [54]:
list(cla.df.columns)

['GDPCQOQIndex',
 'GDPCYOYIndex',
 'GDPCTOT%Index',
 'GPDITOC%Index',
 'GPGSTOC%Index',
 'RGCDCIPIIndex',
 'GDPCUR$Index',
 'GDPCURYIndex',
 'GDPPIQQIndex',
 'GDPCPCECIndex',
 'CPIYOYIndex',
 'CPIXYOYIndex',
 'PCEDEFYIndex',
 'PCECYOYIndex',
 'CPICHNGIndex',
 'CPUPXCHGIndex',
 'FDIDFDMOIndex',
 'FDIDSGMOIndex',
 'FDIUFDYOIndex',
 'FDIUSGYOIndex',
 'USURTOTIndex',
 'USUDMAERIndex',
 'INJCJCIndex',
 'INJCSPIndex',
 'NFPTCHIndex',
 'NFPPCHIndex',
 'USMMMNCHIndex',
 'AWHTOTLIndex',
 'USWHTOTIndex',
 'USEMTOTIndex',
 'USEMNCHGIndex',
 'USERTOTIndex',
 'EMDINP1MIndex',
 'JOLTOPENIndex',
 'JOLTHIRSIndex',
 'JOLTSEPSIndex',
 'CHALYOY%Index',
 'ADPCHNGIndex',
 'USHEYOYIndex',
 'USHETOT%Index',
 'ECIYOYIndex',
 'COSTNFR%Index',
 'PRODNFR%Index',
 'IPYOYIndex',
 'IPCHNGIndex',
 'CPTICHNGIndex',
 'TMNOCHNGIndex',
 'DGNOYOYIndex',
 'DGNOCHNGIndex',
 'MTIBCHNGIndex',
 'MWINCHNGIndex',
 'MGT2TBIndex',
 'MGT2MAIndex',
 'MGT2WHIndex',
 'MGT2WHDUIndex',
 'MGT2REIndex',
 'IPIndex',
 'MTSLRL$Index',
 'AGW

In [89]:
(newdf['LEISTKPIndex'] - newdf['LEISTKPIndex'].shift(periods = 1, axis = 0)).value_counts()

Series([], Name: LEISTKPIndex, dtype: int64)

In [90]:
newdf['LEISTKPIndex']

date
1981-01-01   NaN
1981-01-02   NaN
1981-01-05   NaN
1981-01-06   NaN
1981-01-07   NaN
              ..
2019-10-27   NaN
2019-10-28   NaN
2019-10-29   NaN
2019-10-30   NaN
2019-10-31   NaN
Name: LEISTKPIndex, Length: 11682, dtype: float64

In [83]:
newdf['LEISTKPIndex'].value_counts()

166.39     2
1278.72    2
463.81     2
2754.86    1
2901.50    1
          ..
1918.60    1
925.29     1
992.54     1
848.15     1
188.89     1
Name: LEISTKPIndex, Length: 452, dtype: int64

In [85]:
newdf['LEISTKPIndex']

date
1981-01-01   NaN
1981-01-02   NaN
1981-01-05   NaN
1981-01-06   NaN
1981-01-07   NaN
              ..
2019-10-27   NaN
2019-10-28   NaN
2019-10-29   NaN
2019-10-30   NaN
2019-10-31   NaN
Name: LEISTKPIndex, Length: 11682, dtype: float64

In [62]:
cla.df['LEISTKPIndex'].unique()

array([    nan,  132.97,  128.4 ,  133.19,  134.43,  131.73,  132.28,
        129.13,  129.63,  118.27,  119.8 ,  122.92,  123.79,  117.28,
        114.5 ,  110.84,  116.31,  116.35,  109.7 ,  109.38,  109.65,
        122.43,  132.66,  138.1 ,  139.37,  144.27,  146.8 ,  151.88,
        157.71,  164.1 ,  166.39,  166.96,  162.42,  167.16,  167.65,
        165.23,  164.36,  157.25,  157.44,  157.6 ,  156.55,  153.12,
        151.08,  164.42,  166.11,  164.82,  166.27,  164.48,  171.61,
        180.88,  179.42,  180.62,  184.9 ,  188.89,  192.54,  188.31,
        184.06,  186.18,  197.45,  207.26,  208.19,  219.37,  232.33,
        237.98,  238.46,  245.3 ,  240.18,  245.  ,  238.27,  237.36,
        245.09,  248.61,  264.51,  280.93,  292.47,  289.32,  289.12,
        301.38,  310.09,  329.36,  318.66,  280.16,  245.01,  240.96,
        250.48,  258.13,  265.74,  262.61,  256.12,  270.68,  269.05,
        263.73,  267.97,  277.4 ,  271.02,  276.51,  285.41,  294.01,
        292.71,  302

In [66]:
cla.df['LEISTKPIndex7']

date
1981-01-01   NaN
1981-01-02   NaN
1981-01-05   NaN
1981-01-06   NaN
1981-01-07   NaN
              ..
2019-10-27   NaN
2019-10-28   NaN
2019-10-29   NaN
2019-10-30   NaN
2019-10-31   NaN
Name: LEISTKPIndex7, Length: 11682, dtype: float64

In [56]:
cla.df['LEISTKPIndex90'].interpolate(method = 'cubic', limit_area = 'inside').fillna(method = 'ffill').fillna(method = 'bfill')

ValueError: The number of derivatives at boundaries does not match: expected 3, got 0+0